## Python 爬虫原理(5):JS加密

　　在爬虫过程中，经常给服务器造成压力（比如耗尽CPU，内存，带宽等），为了减少不必要的访问（比如爬虫），网页开发者就发明了反爬虫技术。<br>
　　常见的反爬虫技术有封ip，user_agent，字体库，js加密，验证码（字符验证码，滑动验证码，点触式验证码等）。所谓魔高一尺道高一丈。有反爬虫，就有反反爬虫技术。本文重要讲js加密的破解方法。<br>
　　js加密一般是在请求头或者请求参数加入加密有的字段。爬虫开发者不知道加密的方法，就能够抵挡一些低级爬虫工程师。但是js加密函数或者过程一定是在浏览器完成，也就是一定会把js代码暴露给使用者。通过阅读加密算法，就可以模拟出加密过程，从而达到破解。<br>
### 1、爬虫js破解的方式有：<br>
　　一：把js代码翻译成Python或者java等代码。这个对于大部分的爬虫工程师是有难度的。因为爬虫偏向于后端的处理，很少有爬虫工程师去精通前端的知识，还有一点就是经过加密，混淆，压缩的js代码阅读性很差的。即使是开发三五年的前端工程师，也很少能看懂的。更何况是没有搞过js的爬虫工程师呢？当然也有一些高手除外。<br>
　　例如有道翻译参数加密：<br>
　　r = "" + ((new Date).getTime() + parseInt(10 * Math.random(), 10))    # 所谓的加密的salt（盐）<br>
　　sign：n.md5("fanyideskweb" + t + r + "p09@Bn{h02_BIEe]$P^nG");      # 加密参数sign:Md5加密，第一、四参数为常量，二为翻译的单词，三为加密的盐。<br>
　　二: 执行js代码。可以使用Python的一些第三方库比如 pyv8，或者execjs去执行js代码。这个代码效率很高的。<br>
　　三：使用selenium或者appiun等框架。强烈不建议使用。因为代码的稳定性和效率都很低。<br>

### 2、执行JS代码反反爬虫<br>
1）、**安装相应库：**pip install PyExecJS  　　　　　　# 需要注意， 包的名称：PyExecJS <br>
2）、**PyExecJS的简单使用：**<br>
>import execjs<br>
    **execjs.eval("new Date")**<br>
    返回值为： 2018-04-04T12:53:17.759Z<br>
    execjs.eval("Date.now()")<br>
    返回值为：1522847001080  # 需要注意的是返回值是13位， 区别于python的time.time()（9位）<br>
    
    注意： 个别的JS语句， 用execjs返回的结果跟浏览器环境返回的结果是有区别的<br>
3）、**execjs调用函数：**<br>
  实际生产中处理的js有几百几千行， 不方便贴上来。来看一下源码中给的例子：<br>
  >**ctx = execjs.compile**("""<br>
        function add(x, y) {<br>
                return x + y;<br>
           }<br>
""")<br>
  >**ctx.call("add", 1, 2)**  # 第一个参数 “add” 为JS函数名的字符串， 后边依次为实参<br>
  返回值：3<br>
  
4）、**执行JS的环境管理：**<br>
    a、在windows上不需要其他的依赖便可运行execjs， 也可以调用其他的JS环境<br>
    windows 默认的执行JS的环境<br>
>execjs.get().name<br>
       返回值： JScript<br>
       
    b、在ubuntu下需要安装执行JS环境依赖, 作者的环境为PhantomJS<br>
>execjs.get().name<br>
       返回值： PhantomJS<br>
       
    c、源码中给出， 可执行execjs的环境：<br>
        PyV8           = "PyV8"<br>
        Node           = "Node"<br>
        JavaScriptCore = "JavaScriptCore"<br>
        SpiderMonkey   = "SpiderMonkey"<br>
        JScript        = "JScript"<br>
        PhantomJS      = "PhantomJS"<br>
        SlimerJS       = "SlimerJS"<br>
        Nashorn        = "Nashorn"<br>
    d、Js环境切换：<br>
      # 1. 通过os.environ<br>
      
>os.environ["EXECJS_RUNTIME"] = "Node"   # 环境路径<br>
>execjs.get().name<br>

      # 2. 通过execjs.get 切换<br>
>jscript = execjs.get(execjs.runtime_names.JScript)  　　　　# runtime_names 便是execjs源码中给出的执行环境的。
>node = execjs.get(execjs.runtime_names.Node)<br>
>execjs.runtime_names.xxx  xxx必须在上一节 c 中取<br>

5）、**常见错误处理：**<br>
>raise exceptions.ProgramError(e) execjs._exceptions.ProgramError: ReferenceError: XXX is not defined <br>
>Uncaught TypeError: Cannot read property 'length' of undefined
>Uncaught SyntaxError: Unexpected token ILLEGAL

execjs只是纯粹的js运行环境，没有dom结构，不能像浏览器一样传递调用dom中的一些变量或函数。<br>
这时解决办法是：在js代码字符串中定义上相应的变量对象或函数对象。<br>
>Uncaught SyntaxError: Unexpected token <br>

一种解释是：由于AJAX请求的地址需要登录才能访问，不登录的话会跳转到登录页面，所以返回的是登录页面的HTML代码，导致执行“eval”函数出错（未指出解决办法）；<br>
一种前端解释是：js报错，某个js文件未引入<br>

5）、PyExecJS的缺点之一就是性能。PyExecJS通过文本传递JavaScript运行时，并且速度很慢。另一个缺点是它不完全支持运行时特定的功能。对于某些用例，PyV8可能是更好的选择。<br>
总注：使用execjs的难点并不是在execjs这个库，而是解析JS的过程，因为没有浏览器的环境，没有加密源码的依赖。从成千上万行的JS中择出想要的内容，可能是一段孤零零的JS函数，也可能是从几个JS文件去找出各自找出一段JS代码，并可以通过execjs顺利执行，这并非易事。需要慢慢积累经验。 一旦掌握，便可以提高爬虫的效率，以及代码的健壮性，节省资源！<br>